# Recommendation Engine

In [23]:
# load package
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

In [24]:
# load data
df_movie = pd.read_csv("data/movies_metadata.csv")
df_ratings = pd.read_csv("data/ratings_small.csv")
pd.set_option('display.max_columns', None)
df_movie.head()

/var/folders/5_/39stv43d5_70psn53dfmjwt00000gn/T/ipykernel_10281/2682971138.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movie = pd.read_csv("data/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [26]:
df_movie['movieId'] = pd.to_numeric(df_movie['id'], errors='coerce')
df = pd.merge(df_ratings, df_movie, on='movieId', how='inner')
df_re = df.iloc[:, 0:3]
df_re.head()

,userId,movieId,rating
0,1,1371,2.5
1,4,1371,4.0
2,7,1371,3.0
3,19,1371,4.0
4,21,1371,3.0


## Building the Recommendation Engine

### User based recommendation engine


In [28]:
# build rating matrix
ratings = df_re.groupby(["userId","movieId"])["rating"].mean().unstack().reset_index().set_index('userId')
ratings.head()

movieId   2    3   5   6  11  12  13  14  15  16   17  18  19  20  21  22  24  \
userId                                                                          
1       NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN NaN   
2       NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  5.0 NaN NaN NaN NaN NaN NaN   
3       NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN NaN   
4       NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN NaN   
5       NaN  4.0 NaN NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN NaN   

movieId  25  26  27  28  30  35  38  55  58  59   62  63  64  65  66  68  69  \
userId                                                                         
1       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN NaN   
2       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  3.0 NaN NaN NaN NaN NaN NaN   
3       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN NaN   
4       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN NaN   
5       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN NaN   

movieId  70  71  73  74  76  77  78  79  80  81  82  83  85  86  87  88  89  \
userId                                                                        
1       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
2       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
3       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
4       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
5       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   

movieId  92  93  95  96  97  98  99  100  101  102  103  104  105  107  108  \
userId                                                                        
1       NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2       NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3       NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4       NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5       NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  4.0  NaN  NaN  NaN   

movieId  110  111  112  113  114  116  117  118  121  122  123  124  129  132  \
userId                                                                          
1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2        4.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3        4.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4        NaN  NaN  5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

movieId  135  137  140  141  144  145  146  147  148  149  150  152  153  154  \
userId                                                                          
1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2        NaN  NaN  NaN  NaN  3.0  NaN  NaN  NaN  NaN  NaN  5.0  NaN  4.0  NaN   
3        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4        NaN  NaN  NaN  5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  4.0  NaN   
5        NaN  NaN  NaN  4.0  NaN  NaN  NaN  NaN  NaN  NaN  4.0  NaN  NaN  NaN   

movieId  155  156  157  158  159  160  161  162  163  164  165  166  167  168  \
userId                                                                          
1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2        NaN  NaN  NaN  NaN  NaN  NaN  3.0  NaN  NaN  NaN  3.0  NaN  NaN  3.0   
3        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
4        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

movieId  169  170  171  172  173  174  175  176  177  178  

In [29]:
# Normalize the rating matrix
rating_norm = ratings.subtract(ratings.mean(axis=1), axis = 0)
rating_norm.head()

movieId   2         3   5   6  11  12  13  14  15  16        17  18  19  20  \
userId                                                                        
1       NaN       NaN NaN NaN NaN NaN NaN NaN NaN NaN       NaN NaN NaN NaN   
2       NaN       NaN NaN NaN NaN NaN NaN NaN NaN NaN  1.482759 NaN NaN NaN   
3       NaN       NaN NaN NaN NaN NaN NaN NaN NaN NaN       NaN NaN NaN NaN   
4       NaN       NaN NaN NaN NaN NaN NaN NaN NaN NaN       NaN NaN NaN NaN   
5       NaN  0.084906 NaN NaN NaN NaN NaN NaN NaN NaN       NaN NaN NaN NaN   

movieId  21  22  24  25  26  27  28  30  35  38  55  58  59        62  63  64  \
userId                                                                          
1       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN       NaN NaN NaN   
2       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN -0.517241 NaN NaN   
3       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN       NaN NaN NaN   
4       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN       NaN NaN NaN   
5       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN       NaN NaN NaN   

movieId  65  66  68  69  70  71  73  74  76  77  78  79  80  81  82  83  85  \
userId                                                                        
1       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
2       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
3       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
4       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
5       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   

movieId  86  87  88  89  92  93  95  96  97  98  99  100  101  102  103  \
userId                                                                    
1       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN   
2       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN   
3       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN   
4       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN   
5       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN   

movieId       104  105  107  108       110  111      112  113  114  116  117  \
userId                                                                         
1             NaN  NaN  NaN  NaN       NaN  NaN      NaN  NaN  NaN  NaN  NaN   
2             NaN  NaN  NaN  NaN  0.482759  NaN      NaN  NaN  NaN  NaN  NaN   
3             NaN  NaN  NaN  NaN  0.460000  NaN      NaN  NaN  NaN  NaN  NaN   
4             NaN  NaN  NaN  NaN       NaN  NaN  0.72807  NaN  NaN  NaN  NaN   
5        0.084906  NaN  NaN  NaN       NaN  NaN      NaN  NaN  NaN  NaN  NaN   

movieId  118  121  122  123  124  129  132  135  137  140       141       144  \
userId                                                                          
1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN       NaN       NaN   
2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN       NaN -0.517241   
3        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN       NaN       NaN   
4        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.728070       NaN   
5        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.084906       NaN   

movieId  145  146  147  148  149       150  152       153  154  155  156  157  \
userId                                                                          
1        NaN  NaN  NaN  NaN  NaN       NaN  NaN       NaN  NaN  NaN  NaN  NaN   
2        NaN  NaN  NaN  NaN  NaN  1.482759  NaN  0.482759  NaN  NaN  NaN  NaN   
3        NaN  NaN  NaN  NaN  NaN       NaN  NaN       NaN  NaN  NaN  NaN  NaN   
4        NaN  NaN  NaN  NaN  NaN       NaN  NaN -0.271930  NaN  NaN  NaN  NaN   
5        NaN  NaN  NaN  NaN  NaN  0.084906  NaN       NaN  NaN  NaN  NaN  NaN   

movieId  158  159  160       161  162  163  164       165  166  167       168  \
userId              

In [30]:
# User similarity matrix using Pearson correlation
user_similarity = rating_norm.T.corr(method='pearson')
user_similarity.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,NaN,0.693375,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.644094,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.970725,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.490990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.862662,NaN,NaN,NaN,NaN,NaN,NaN,-0.733333,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.693375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.962250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.944911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [33]:
# find users with similar ratings
target_user = 1
product_id = 2
target_user_similarity = user_similarity.loc[[target_user]].dropna(axis=1, how='all').reset_index()
target_user_similarity.drop(target_user_similarity.iloc[:, 0:1], inplace=True, axis=1)
target_user_similarity = target_user_similarity.T
target_user_similarity.columns = ["similarity to the targeted customer"]
target_user_similarity.reset_index()
target_user_similarity.drop(target_user, axis=0, inplace=True)
target_user_similarity.head()

,similarity to the targeted customer
userId,
4,0.693375
7,-1.000000
15,0.644094
19,1.000000
30,-1.000000


In [35]:
# get rating to target product based for similar users
target_item_ratings = rating_norm.loc[:,product_id].dropna(axis=0, how='all').reset_index()
target_item_ratings.columns = ["userId","rating to the targeted product"]
target_item_ratings.head()

,userId,rating to the targeted product
0,15,-0.798230
1,19,-0.471774
2,30,-1.754651
3,32,0.350000
4,41,-0.370588


In [37]:
# join target_ratings and target_user_similarity
ratings_with_user_sim = target_item_ratings.merge(target_user_similarity, on = "userId", how = "inner")
top5_users = ratings_with_user_sim.sort_values(by = "similarity to the targeted customer", ascending = False).iloc[0:5,]
# top 5 users who are most similar to the target user and have rated the target product
top5_users

,userId,rating to the targeted product,similarity to the targeted customer
17,561,-0.577778,1.000000
13,466,-0.776923,1.000000
1,19,-0.471774,1.000000
11,312,-1.322785,1.000000
3,41,-0.370588,0.970725


In [50]:
#Calculate the predicted rating using weighted average of similarity
predicted_rating = round(np.average(top5_users['rating to the targeted product'],
                                    weights=top5_users['similarity to the targeted customer']), 2)
print(f'The user-based predicted rating for product {product_id} by user {target_user} is {predicted_rating}' )

The user-based predicted rating for product 2 by user 1 is -0.71


### Item based recommendation engine

In [39]:
# Product similarity matrix using Pearson correlation
item_similarity = rating_norm.corr(method='pearson')
item_similarity.head()

movieId         2         3         5         6        11        12        13  \
movieId                                                                         
2        1.000000 -0.100692 -0.313484 -0.305460  0.325447 -0.234125 -0.597517   
3       -0.100692  1.000000  0.309948 -0.220351  0.456561  0.538085 -0.758534   
5       -0.313484  0.309948  1.000000  0.268590 -0.142193  0.021809 -0.726284   
6       -0.305460 -0.220351  0.268590  1.000000  0.000666  0.166841 -1.000000   
11       0.325447  0.456561 -0.142193  0.000666  1.000000  0.080683  0.034895   

movieId        14        15        16        17        18        19        20  \
movieId                                                                         
2       -0.194167  0.158825 -0.439108  0.086328  0.392105 -0.110902  0.376390   
3       -0.533936  0.658186 -0.146618 -0.075178 -0.155647  0.430196 -0.991060   
5       -0.016314  0.213012  0.143459 -0.232439 -0.381543 -0.005829  0.332801   
6       -0.261603  0.999473 -0.030674 -0.209969  0.345445 -0.039015 -0.285164   
11      -0.245093  0.194180 -0.089756 -0.119501 -0.569764  0.116454 -0.051017   

movieId        21        22        24        25   26        27        28  \
movieId                                                                    
2       -0.184776  0.205194  0.004317 -0.298525  NaN -0.819137  0.456191   
3        0.076889  0.102305  0.844593 -0.165751  NaN  0.983896  1.000000   
5        0.203961 -0.421052  0.872323 -0.308409  1.0  1.000000  1.000000   
6       -0.064817 -0.160591 -0.469532 -0.150628  NaN -0.626616  0.241464   
11       0.263152  0.192376 -0.392105 -0.230834 -1.0 -0.614326 -0.499201   

movieId        30        35   38        55        58  59        62        63  \
movieId                                                                        
2        1.000000 -0.466895  NaN  1.000000 -0.762573 NaN  0.022854 -0.986865   
3        0.815569 -0.551611  1.0       NaN  0.121578 NaN  0.151693  0.498630   
5       -1.000000 -0.868297  1.0       NaN -0.216932 NaN  0.171575       NaN   
6       -0.761958  0.968651  NaN -0.735821 -0.339980 NaN  0.105091  0.993648   
11      -0.556490 -0.554108  NaN       NaN -0.036681 NaN  0.333373  1.000000   

movieId        64        65        66        68        69        70        71  \
movieId                                                                         
2        0.382984  0.344870 -0.952651       NaN -0.650265 -0.049544 -0.831139   
3        0.290816  0.403448  0.999998 -1.000000  0.777300  0.731600 -1.000000   
5        0.804328 -0.035142  1.000000       NaN  0.429911 -0.473861 -1.000000   
6        0.304701  0.262776 -1.000000 -0.202716 -0.494362  0.185536  0.190561   
11      -0.249904  0.462148  1.000000  0.672749  0.429773  0.233955  0.188760   

movieId        73        74        76  77        78        79  80        81  \
movieId                                                                       
2       -0.649959  0.147179 -0.601711 NaN  1.000000 -0.514944 NaN -0.604458   
3             NaN  0.596426 -0.116142 NaN -0.633781 -0.041591 NaN  0.785314   
5        1.000000  0.568039 -0.359025 NaN -0.853482  0.445680 NaN  0.708574   
6        0.733499 -0.213047 -0.544839 NaN -0.489436  0.356349 NaN  0.086149   
11      -0.034336  0.067659 -0.472257 NaN -1.000000 -0.113744 NaN -0.608647   

movieId        82   83        85        86   87        88        89        92  \
movieId                                                                         
2       -1.000000  NaN  0.178351 -0.377488 -1.0  0.282019  0.382495  1.000000   
3             NaN  NaN  0.869719 -0.505464 -1.0  0.048322 -0.357620 -0.521550   
5        0.517756  NaN  0.952921 -0.281494 -1.0 -0.522349 -0.264777 -0.015040   
6        0.088491  1.0  0.104697  0.390457  NaN  0.527761  0.554489 -0.918201   
11       0.589201  NaN -0.021744  0.333637  1.0  0.433356  0.364449 -0.696271   

movieId        93        95  96        97  98        99       100       101  \
m

In [45]:
# find similar items
target_item_similarity_score = item_similarity.loc[[product_id]].dropna(axis=1, how='all').reset_index()
target_item_similarity_score.drop(target_item_similarity_score.iloc[:,0:1], inplace= True,axis = 1)
target_item_similarity_score = target_item_similarity_score.T.reset_index()
target_item_similarity_score.columns = ["ProductId","similarity to the targeted product"]
target_item_similarity_score.drop(target_item_similarity_score[target_item_similarity_score['ProductId'] == product_id].index, inplace=True)
target_item_similarity_score.head(10)

,ProductId,similarity to the targeted product
1,3,-0.100692
2,5,-0.313484
3,6,-0.305460
4,11,0.325447
5,12,-0.234125
6,13,-0.597517
7,14,-0.194167
8,15,0.158825
9,16,-0.439108
10,17,0.086328


In [46]:
# select items that have been rated by the targeted user
target_user_rated = rating_norm.loc[[target_user]].dropna(axis=1, how='all').reset_index()
target_user_rated.drop(target_user_rated.iloc[:, 0:1], inplace=True, axis=1)
target_userid_rated = target_user_rated.T
target_userid_rated.reset_index(inplace=True)
target_userid_rated .columns = ["ProductId","target_user_rating"]
# target_userid_rated.reset_index(inplace=True)
target_userid_rated.head(10)

,ProductId,target_user_rating
0,1371,0.166667
1,1405,-1.333333
2,2105,1.666667
3,2193,-0.333333
4,2294,-0.333333
5,2455,0.166667


In [47]:
# join target_user_rated and target_item_similarity_score
ratings_with_item_sim = target_userid_rated.merge(target_item_similarity_score, on= "ProductId", how = "inner")
ratings_with_item_sim.head(10)

,ProductId,target_user_rating,similarity to the targeted product
0,1371,0.166667,-0.390476
1,1405,-1.333333,0.195115
2,2105,1.666667,-0.056787
3,2193,-0.333333,0.231872
4,2294,-0.333333,0.578406
5,2455,0.166667,0.286024


In [48]:
# Select top 5 products that are most similar to the target product and have been rated by the target user
top5_items = ratings_with_item_sim.sort_values(by="similarity to the targeted product", ascending= False).iloc[0:5,:]
top5_items

,ProductId,target_user_rating,similarity to the targeted product
4,2294,-0.333333,0.578406
5,2455,0.166667,0.286024
3,2193,-0.333333,0.231872
1,1405,-1.333333,0.195115
2,2105,1.666667,-0.056787


In [51]:
#Calculate the predicted rating using weighted average of similarity
predicted_rating = round(np.average(top5_items['target_user_rating'],
                                    weights=top5_items['similarity to the targeted product']), 2)
print(f'The item-based predicted rating for product {product_id} by user {target_user} is {predicted_rating}' )

The item-based predicted rating for product 2 by user 1 is -0.47
